<a href="https://colab.research.google.com/github/Oliver-Shariff/Pro-Image-Sports-ETL-Scripts/blob/main/Pro_Image_Sports_Commission_Calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Commission Calc
"""
By employee
$1 per jersey w/ a gross selling price >= 108
$2 per embroidery
10% of profit from Memrobillia sales (subtotal - cost)* .1
"""

In [ ]:
from google.colab import files
import pandas as pd

mem_commission_rate = .1 # 10% of profit

In [ ]:
# Load the XLSX File
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

In [ ]:
# Read file and keep necessary columns
df = pd.read_excel(file_path,skiprows=5)
df = df[['Employee','Category', 'Selling price, $','Quantity','Tax, $', 'Gross sales, $','Discount, $', 'Cost, $']]
#print(df)

In [ ]:
# Remove clearance jerseys (Replica) from calculation
df = df[~((df['Category'] == 'REPLICA') & ((df['Gross sales, $'] - df['Tax, $']) < 99.99))]

#display(df)

In [ ]:
# Remove all categories that don't recieve
df = df[df['Category'].isin(['HAT EMBROIDERY', 'MEMORABILIA/PRINTS', 'REPLICA'])]
#display(df)

In [ ]:
# Calculate comission made on Memorbillia/prints (10% of profit)
df['Mem_Comission'] = (df['Gross sales, $'] - df['Tax, $']- (df['Cost, $']*df['Quantity']))*mem_commission_rate
#display(df)

In [ ]:
# Calculate total quantity for REPLICA (Jerseys) by employee
replica_qty = df[df['Category'] == 'REPLICA'].groupby('Employee')['Quantity'].sum().reset_index()
replica_qty.rename(columns={'Quantity': 'Total QTY Replica'}, inplace=True)

# Calculate total quantity for HAT EMBROIDERY by employee
embroidery_qty = df[df['Category'] == 'HAT EMBROIDERY'].groupby('Employee')['Quantity'].sum().reset_index()
embroidery_qty.rename(columns={'Quantity': 'Total QTY Embroidery'}, inplace=True)

# Calculate sum of Mem_Comission for MEMORABILIA/PRINTS by employee
mem_commission_sum = df[df['Category'] == 'MEMORABILIA/PRINTS'].groupby('Employee')['Mem_Comission'].sum().reset_index()
mem_commission_sum.rename(columns={'Mem_Comission': 'SUM Mem_Comission'}, inplace=True)

# Merge the dataframes
employee_summary = replica_qty.merge(embroidery_qty, on='Employee', how='outer').merge(mem_commission_sum, on='Employee', how='outer')

# Fill NaN values with 0
employee_summary.fillna(0, inplace=True)

# Add the 'TOTAL' column, $1 for jerseys, $2 for emboridery, and sum of mem_comission
employee_summary['TOTAL'] = ((employee_summary['Total QTY Replica']) + (employee_summary['Total QTY Embroidery']*2) + employee_summary['SUM Mem_Comission']).round(2)


display(employee_summary)